In [1]:
from visio_gptq.model import GPTQModel
model = GPTQModel(model_name="anon8231489123/vicuna-13b-GPTQ-4bit-128g", device="cuda", wbits=4, groupsize=128)

/home/marcojoao/mambaforge/envs/vicuna/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model ...


/home/marcojoao/mambaforge/envs/vicuna/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/marcojoao/mambaforge/envs/vicuna/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/marcojoao/mambaforge/envs/vicuna/lib/python3.10/site-packages/torch/storage.py:899: UserWarning: TypedStorage is deprecated. It will

Done.


In [2]:
from langchain.llms.base import LLM
from typing import Optional, List

class GPTQModelLLM(LLM):
    model: GPTQModel
    params: dict = {}

    @property
    def _llm_type(self) -> str:
        return "gptq_model_llm"

    def callbackFunc(self,txt):
        print(txt, end=" ", flush=True)

    def _call(self, prompt:str, stop: Optional[List[str]] = None) -> str:
        current_stop = self.params.get("stop", [])
        if stop is not None:
            current_stop.extend(stop)
            self.params["stop"] = current_stop

        conv = model.inference(prompt=prompt, **self.params)
        if isinstance(conv, str):
            return conv
        return "Error"

In [11]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
llm = GPTQModelLLM(model=model, params={"temperature": 0.0, "stop":["Question:", "###"]})
DEFAULT_PROMPT = PromptTemplate(
    input_variables=["history","input"],
    template="The following is a friendly conversation between a human and an AI. The AI is talkative and provides direct answers based only on context. If the AI does not know the answer to a question, it truthfully says it does not know."
            "Current conversation:"
            "{history}"
            "### Human: {input}"
            "### AI:")
conv = ConversationChain(llm=llm, prompt=DEFAULT_PROMPT, verbose=False)

In [12]:
result = conv.predict(input="What is the capital of Portugal?")
print(result)

The capital of Portugal is Lisbon.

Lisbon is the largest city in Portugal, and it is located on the Atlantic coast in the western part of the country. It is a major economic, financial, and cultural center, and it is known for its historic architecture, museums, and cultural events. Lisbon is also a popular tourist destination, and it is famous for its picturesque neighborhoods, traditional cuisine, and vibrant nightlife.


In [13]:
result = conv.predict(input="and Spain?")
print(result)

The capital of Spain is Madrid.

Madrid is the largest city in Spain and it is located in the center of the country. It is a major economic, political, and cultural center, and it is known for its vibrant atmosphere, cultural attractions, and historical landmarks. Madrid is home to some of the world's most famous museums, including the Prado Museum, which houses an extensive collection of European art, and the Reina Sofia Museum, which is dedicated to modern and contemporary art. The city is also famous for its lively nightlife, delicious food, and beautiful architecture.


In [14]:
result = conv.predict(input="What was my first question?")
print(result)

Your first question was: "What is the capital of Portugal?"
